Back to the main [Index](index.ipynb) <a id="top"></a>

# Post-processing DFPT calculations with the DDB file

This notebook explains how to use AbiPy and the DDB file produced by Abinit to analyze:

* foo
* bar

In the last part, we discuss how to use the `DdbRobot` to analyze multiple DDB
files and perform typical convergence studies.

## Table of Contents

* [How to create a Ddbfile object](#How-to-create-a-DdbFile-object)
* [Invoking Anaddb from the DdbFile object](#Invoking-Anaddb-from-the-DdbFile-object)

## Suggested references

* [Dynamical matrices, Born effective charges, dielectric permittivity tensors, and interatomic force constants from density-functional perturbation theory](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.55.10355)
* [Phonons and related crystal properties from density-functional perturbation theory](https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.73.515)

AbiPy, [pymatgen](http://pymatgen.org/) and [fireworks](https://materialsproject.github.io/fireworks/)
have been used by [Petretto et al](https://www.sciencedirect.com/science/article/pii/S0927025617307243)
to compute the vibrational properties of more than 1500 compounds with Abinit.
The results are available on the [materials project website](https://materialsproject.org/).
The results for this phase of MgO are available at <https://materialsproject.org/materials/mp-1009129/>

To fetch the DDB file from the materials project database and build a `DdbFile` object, use:

```python
    ddb = abilab.DdbFile.from_mpid("mp-1009129")
```

<hr>
Remember to set the `PMG_MAPI_KEY` in your ~/.pmgrc.yaml as described 
[here](http://pymatgen.org/usage.html#setting-the-pmg-mapi-key-in-the-config-file).

## How to create a DdbFile object 
[[back to top](#top)]

Let us start by importing the basic AbiPy modules we have already used in the other examples:

In [1]:
from __future__ import division, print_function, unicode_literals

import warnings 
warnings.filterwarnings("ignore")  # Ignore warnings

from abipy import abilab
abilab.enable_notebook() # This line tells AbiPy we are running inside a notebook

import abipy.data as abidata

# This line configures matplotlib to show figures embedded in the notebook, 
# instead of popping up a new window. 
%matplotlib notebook

To open a DDB file, use the high-level interface provided by abiopen:

In [2]:
#ddb_filepath = abidata.ref_file("mp-1009129-9x9x10q_ebecs_DDB")
#ddb = abilab.abiopen(ddb_filepath)

A DdbFile has a structure object:

In [3]:
#print(ddb.structure)  # Lengths in Angstrom.

If you are a terminal aficionado, remember that one can use the 
[abiopen.py](http://abinit.github.io/abipy/scripts/abiopen.html) script 
to open the DDB file directly from the shell and generate a jupyter notebook with the `-nb` option.
For a quick visualization script use [abiview.py](http://abinit.github.io/abipy/scripts/abiview.html).

## Invoking Anaddb from the DdbFile object 
[[back to top](#top)]

The `DdbFile` object provides specialized methods to invoke anaddb and 
compute important physical properties such as the phonon band structure, the phonon density of states etc.
All these methods have a name that begins with the `ana*` prefix followed by a verb (`anaget`, `anacompare`).
These specialized methods 

- build the anaddb input file 
- run anaddb
- parse the netcdf files produced by the Fortran code
- build and return [AbiPy objects](http://abinit.github.io/abipy/api/dfpt_api.html) that can be used to plot/analyze the data.

Note that in order to run anaddb from AbiPy, you need a manager.yml with configuration options.
For further details, please consult the 
[TaskManager documentation](http://abinit.github.io/abipy/workflows/taskmanager.html).

The python API is flexible and exposes several anaddb input variables.
The majority of the arguments have default values covering the most common cases
so that you will need to specify these arguments explicitly only if the default behavior does not suit your needs.
The most important parameters to remember are:

* **ndivsm**: Number of divisions used for the smallest segment of the high-symmetry q-path
* **nqsmall**: Defines the q-mesh for the phonon DOS in terms of
     the number of divisions used to sample the smallest reciprocal lattice vector. 0 to disable DOS computation
* **lo_to_splitting**: Activate the computation of the frequencies in the $q\rightarrow 0$ with the inclusion of the non-analytical term (requires **dipdip** 1 and DDB with $Z^*_{\kappa,\alpha\beta}$ and $\epsilon^{\infty}_{\alpha\beta}$).

The high-symmetry q-path is automatically selected assuming the structure
fulfills the conventions introduced by [Setyawan and Curtarolo](https://arxiv.org/abs/1004.2974)
but you can also specify your own q-path if needed.

## Plotting phonon bands and DOS
[[back to top](#top)]

To compute phonon bands and DOS, use:

In [4]:
# Call anaddb to compute phonon bands and DOS. Return PHBST and PHDOS netcdf files.
#phbstnc, phdosnc = ddb.anaget_phbst_and_phdos_files(
#    ndivsm=20, nqsmall=20, lo_to_splitting=True, asr=2, chneut=1, dipdip=1, dos_method="tetra")

# Extract phbands and phdos from the netcdf object.
#phbands = phbstnc.phbands
#phdos = phdosnc.phdos

Let us have a look at the high symmetry q-path automatically selected by AbiPy with:

## Using `DdbRobot` to perform convergence studies
[[back to top](#top)]

A `DddRobot` receives a list of DDB files and provides methods 
to construct [pandas dataframes](https://pandas.pydata.org/pandas-docs/stable/10min.html)
and analyze the results of multiple calculations.
DDbRobots, in particular, are extremely useful to study the convergence of the phonon frequencies with respect to some computational parameters e.g. the number of k-points and the electronic smearing in metallic systems.

In this example, we are interested in the effect of the k-point sampling and the smearing parameter
on the vibrational properties of magnesium diboride.
$MgB_2$ is a metallic system with a critical temperature of 39 K that is the highest among conventional 
(phonon-mediated) superconductors. 
We use precomputed DDB files obtained by running GS+DFPT calculations with different values 
of `nkpt` and `tsmear`. 

Let's build our `DdbRobot` object with:

In [5]:
#import os
#paths = [
#    #"mgb2_444k_0.01tsmear_DDB",
#    #"mgb2_444k_0.02tsmear_DDB",
#    #"mgb2_444k_0.04tsmear_DDB",
#    "mgb2_888k_0.01tsmear_DDB",
#    "mgb2_888k_0.02tsmear_DDB",
#    "mgb2_888k_0.04tsmear_DDB",
#    "mgb2_121212k_0.01tsmear_DDB",
#    "mgb2_121212k_0.02tsmear_DDB",
#    "mgb2_121212k_0.04tsmear_DDB",
#]
#
#paths = [os.path.join(abidata.dirpath, "refs", "mgb2_phonons_nkpt_tsmear", f) for f in paths]
#
#robot = abilab.DdbRobot.from_files(paths)
#robot

The [abicomp.py](http://abinit.github.io/abipy/scripts/abicomp.html)
script provides a command line interface to build robots from a list of files/directories
given as arguments


The DDB files are now stored in the robot with a label constructed from the file path.
These labels, however, are not very informative. In principle we would like to have a label
that reflects the value of `(nkpt, tsmear)` also because these labels
will be used to generate the labels in our plots.

Let's fix it with a function that recomputes the labels from the metadata available in ddb.header:

In [6]:
#function = lambda ddb: "nkpt: %s, tsmear: %.2f" % (ddb.header["nkpt"], ddb.header["tsmear"])
#robot.remap_labels(function);
#robot

We are usually interested in the convergence behavior with respect to one 
or two parameters of the calculations. 
Let's build a pandas dataframe with the most important parameters extracted from the DDB header:

In [7]:
#robot.get_params_dataframe()

Now we tell the robot to invoke anaddb to compute the phonon bands for all DDB files.
Since we are not interested in the phonon DOS, ``nqsmall`` is set to 0

[[back to top](#top)]